In [4]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import ast

In [5]:
# There are occasions where the template requires non-CO2 reported as CO2eq but in aggregate form
#this modifies the original report SATIMGE numbers for non-CO2 entries with the CO2eq GHP from an emissions csv

In [33]:
path_Report = "input/report.csv" # add/replace file path here or filename if in same directory
path_EmissionsFactors = "input/EmissionsFactors.csv" # add/replace file path here or filename if in same directory
path_Output = "input/report_modified.csv" # add/replace file path here or filename if in same directory

# Read in the DataFrames
report_df = pd.read_csv(path_Report, low_memory=False)

# Replace 'Eps' values with 0
report_df['SATIMGE'] = report_df['SATIMGE'].replace('Eps', 0)
report_df['SATIMGE'] = report_df['SATIMGE'].astype(float)

print(report_df.head(10))

  Process Commodity  Year   Scenario Indicator     SATIMGE
0    I2A3    ACTGRP  2017  Reference       CO2  120.892605
1    I2A3    ACTGRP  2018  Reference       CO2  122.449378
2    I2A3    ACTGRP  2019  Reference       CO2  122.317940
3    I2A3    ACTGRP  2020  Reference       CO2  114.200414
4    I2A3    ACTGRP  2021  Reference       CO2  118.106551
5    I2A3    ACTGRP  2022  Reference       CO2  120.599494
6    I2A3    ACTGRP  2023  Reference       CO2  120.810284
7    I2A3    ACTGRP  2024  Reference       CO2  122.043442
8    I2A3    ACTGRP  2025  Reference       CO2  123.359076
9    I2A3    ACTGRP  2026  Reference       CO2  125.303603


In [35]:
check_val = report_df[report_df['Indicator'] == 'N2O']
check_val

,Process,Commodity,Year,Scenario,Indicator,SATIMGE
34,I2B2,ACTGRP,2019,Reference,N2O,0.944000
35,I2B2,ACTGRP,2020,Reference,N2O,0.930232
36,I2B2,ACTGRP,2021,Reference,N2O,0.911498
37,I2B2,ACTGRP,2022,Reference,N2O,0.930738
38,I2B2,ACTGRP,2023,Reference,N2O,0.932365
...,...,...,...,...,...,...
128403,XIPPCOA,INDCOA,2047,Reference,N2O,0.036091
128408,XIPPCOA,INDCOA,2048,Reference,N2O,0.037905
128413,XIPPCOA,INDCOA,2049,Reference,N2O,0.039719
128418,XIPPCOA,INDCOA,2050,Reference,N2O,0.041533


In [36]:
#ADJUST THE EMISSIONS TO CO2EQ:

# set the emissions factors
emf_df = pd.DataFrame({'Indicator': ['CH4','N2O','C2F6','CF4'],
                    'E_factor': [28, 265,11100,6630]})

# Merge the two DataFrames based on the 'Indicator' column
merged_df = report_df.merge(emf_df, on='Indicator', how='left')

# Fill missing values in 'Multiplier' column with 1
merged_df['E_factor'].fillna(1, inplace=True)

# Multiply the values from 'SATIMGE' by the 'E_factor' column
merged_df['Result'] = merged_df['SATIMGE'] * merged_df['E_factor']

# Optionally, drop the 'Multiplier' column or rename columns
merged_df = merged_df.drop('SATIMGE', axis=1)
merged_df = merged_df.drop('E_factor', axis=1)
#rename to new SATIMGE
merged_df = merged_df.rename(columns={'Result': 'SATIMGE'})



,Process,Commodity,Year,Scenario,Indicator,SATIMGE
34,I2B2,ACTGRP,2019,Reference,N2O,250.160000
35,I2B2,ACTGRP,2020,Reference,N2O,246.511503
36,I2B2,ACTGRP,2021,Reference,N2O,241.547027
37,I2B2,ACTGRP,2022,Reference,N2O,246.645499
38,I2B2,ACTGRP,2023,Reference,N2O,247.076600
...,...,...,...,...,...,...
128403,XIPPCOA,INDCOA,2047,Reference,N2O,9.564246
128408,XIPPCOA,INDCOA,2048,Reference,N2O,10.044932
128413,XIPPCOA,INDCOA,2049,Reference,N2O,10.525618
128418,XIPPCOA,INDCOA,2050,Reference,N2O,11.006304
